# Классификация комментариев

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В нашем распоряжении датасет с комментариями и отметкой об их негативном или позитивном содержании.

**Задача** - обучить модель классифицировать комментарии на позитивные и негативные.

**Описание данных**

Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

## Подготовка и обучение

In [1]:
import numpy as np
import pandas as pd

import re

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
w = WordNetLemmatizer()
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.utils import shuffle

from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer, accuracy_score, roc_auc_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\leoci\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leoci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загружаем данные

In [2]:
df = pd.read_csv('C:/Users/leoci/Downloads/toxic_comments.csv')

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


Датасет содержит 159571 строк и 2 столбца. В первом столбце текст, во втором отметка о его токсичности.

Лемматизируем текст, сохраним лемматизированый текст в отдельный столбце.

In [4]:
def lemmatize(text):
    clear_text = " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())
    lemm_list = w.lemmatize(clear_text)
    lemm_text = "".join(lemm_list)
    return lemm_text
df['lemm_text'] = df['text'].apply(lemmatize)

Выделим из датасета целевой признак. Поделим датасет на обучающую и тестовую и валидационную выборки. Зададим размер тестовой выборки 20% от данных.

In [5]:
X = df.drop('toxic', axis=1) 
y = df['toxic'] 
x_pre_train, x_test, y_pre_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
x_train, x_valid, y_train, y_valid = train_test_split(x_pre_train, y_pre_train, test_size=0.25, random_state=123)


Создадим корпусы текстов в обучающей, валидационной и тестовой выборках. Выведем на экран размеры получившихся выборок

In [6]:
corpus_x_train = x_train['lemm_text'].values.astype('U')
corpus_x_test = x_test['lemm_text'].values.astype('U')
corpus_x_valid = x_valid['lemm_text'].values.astype('U')

In [7]:
print(corpus_x_train.shape)
print(y_train.shape)
print(corpus_x_test.shape)
print(y_test.shape)
print(corpus_x_valid.shape)
print(y_valid.shape)

(95742,)
(95742,)
(31915,)
(31915,)
(31914,)
(31914,)


Посчитаем TF-IDF для корпуса текстов. Зададим TF-IDF как признаки для модели.

In [8]:
stopwords = set(nltk_stopwords.words('english'))
count = TfidfVectorizer(stop_words=stopwords)
x_train = count.fit_transform(corpus_x_train)
x_test = count.transform(corpus_x_test) 
x_valid = count.transform(corpus_x_valid) 

In [9]:
print(x_train.shape)
print(x_test.shape)
print(x_valid.shape)

(95742, 125661)
(31915, 125661)
(31914, 125661)


## Обучение моделей

### Логистическая регрессия

Проверим сбалансированность классов

In [10]:
print(y_train.value_counts())

0    86040
1     9702
Name: toxic, dtype: int64


Наблюдается явный дисбаланс классов. При обучении логистической регрессии укажем параметр class_weight = 'balanced'

In [11]:
%%time

model1 = LogisticRegression(solver='saga', class_weight='balanced')
model1.fit(x_train, y_train)


Wall time: 8.03 s


C:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(class_weight='balanced', solver='saga')

Проверим качество модели при различных порогах

In [12]:
prob_valid = model1.predict_proba(x_valid)
prob_one_valid = prob_valid[:, 1]

best_threshold = 0
best_f1 = 0
best_pred1 = []
for threshold in np.arange(0.1, 0.95, 0.01):
    pred1 = prob_one_valid > threshold
    f1 = f1_score(y_valid, pred1)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_pred1 = pred1
    print("Порог = {:.2f} | F1_score = {:.5f}".format(threshold, f1))

print('')
print("Оптимальный порог = {:.2f} | Наилучший F1_score = {:.5f}".format(best_threshold, best_f1))

Порог = 0.10 | F1_score = 0.36119
Порог = 0.11 | F1_score = 0.38000
Порог = 0.12 | F1_score = 0.39755
Порог = 0.13 | F1_score = 0.41493
Порог = 0.14 | F1_score = 0.43076
Порог = 0.15 | F1_score = 0.44663
Порог = 0.16 | F1_score = 0.46376
Порог = 0.17 | F1_score = 0.47981
Порог = 0.18 | F1_score = 0.49540
Порог = 0.19 | F1_score = 0.51017
Порог = 0.20 | F1_score = 0.52547
Порог = 0.21 | F1_score = 0.54122
Порог = 0.22 | F1_score = 0.55533
Порог = 0.23 | F1_score = 0.56676
Порог = 0.24 | F1_score = 0.58223
Порог = 0.25 | F1_score = 0.59301
Порог = 0.26 | F1_score = 0.60443
Порог = 0.27 | F1_score = 0.61652
Порог = 0.28 | F1_score = 0.62638
Порог = 0.29 | F1_score = 0.63471
Порог = 0.30 | F1_score = 0.64467
Порог = 0.31 | F1_score = 0.65316
Порог = 0.32 | F1_score = 0.66021
Порог = 0.33 | F1_score = 0.66854
Порог = 0.34 | F1_score = 0.67649
Порог = 0.35 | F1_score = 0.68433
Порог = 0.36 | F1_score = 0.69016
Порог = 0.37 | F1_score = 0.69582
Порог = 0.38 | F1_score = 0.70295
Порог = 0.39 |

Проверим модель на тестовой выборке при полученном оптимальном пороге.

In [13]:
prob_test = model1.predict_proba(x_test)
prob_one_test = prob_test[:, 1]

pred1_test = prob_one_test > best_threshold
f1 = f1_score(y_test, pred1_test)
    
print("порог = {:.2f} | F1_score = {:.5f}".format(best_threshold, f1))

порог = 0.67 | F1_score = 0.77729


In [14]:
print(f'F1 = {f1_score(y_test, pred1_test):.4}')
print(f'Accuracy при полученном оптимальном пороге = {accuracy_score(y_test, pred1_test):.4}')
print(f'ROC-AUC score при оптимальном пороге = {roc_auc_score(y_test, prob_one_test):.4}')

F1 = 0.7773
Accuracy при полученном оптимальном пороге = 0.9574
ROC-AUC score при оптимальном пороге = 0.9694


Логистическая регрессия быстро обучается и выдаёт достаточную F1 даже при базовом пороге (при пороге 0.5 F1 = 0.752). После подбора оптимального порога удалось довести значение F1 до 0.777

### RandomForestClassifier

In [15]:
%%time

model2 = RandomForestClassifier(random_state=123)
params2 = {'n_estimators' : np.arange(10,25,5)}
grid = GridSearchCV(model2, params2, scoring='f1', cv=2)
grid.fit(x_train, y_train)
print('Комбинация параметров, которая дает лучший результат:', grid.best_params_)
print(f'Максимальный F1: {grid.best_score_:.4f}')

Комбинация параметров, которая дает лучший результат: {'n_estimators': 15}
Максимальный F1: 0.6793
Wall time: 6min 30s


In [16]:
pred2 = grid.predict(x_valid)
print(f'F1 на валидационной выборке = {f1_score(y_valid, pred2):.4}')

pred2 = grid.predict(x_test)
print(f'F1 на тестовой выборке = {f1_score(y_test, pred2):.4}')

F1 на валидационной выборке = 0.7089
F1 на тестовой выборке = 0.7104


Случайный лес обучается значительно дольше логистической регрессии, а получаемый результат оказывается хуже.

### LGBMClassifier

In [17]:
%%time

model3 = LGBMClassifier(random_state=123, class_weight='balanced')
model3.fit(x_train, y_train)

Wall time: 14.8 s


LGBMClassifier(class_weight='balanced', random_state=123)

In [18]:
pred3 = model3.predict(x_valid)
print(f'F1 на валидационной выборке = {f1_score(y_valid, pred3):.4}')

pred3 = model3.predict(x_test)
print(f'F1 на тестовой выборке = {f1_score(y_test, pred3):.4}')

F1 на валидационной выборке = 0.7376
F1 на тестовой выборке = 0.7302


LGBM обучается достаточно быстро, но точность хуже чем у логистической регрессии

## Выводы

Задача была решена, было получено требуемое значение метрики.

В ходе данной работы были обучены различные модели. Наилучшее значение F1_score достигается при использовании модели LogisticRegression, при условии подбора порога. Так же, данная модель является одной из самых быстрообучаемых.